## **Tech Challenge**

### **Projeto**

Entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença, utilizando o estudo do PNAD-COVID 19 do IBGE para termos respostas ao problema proposto, pois são dados confiáveis, porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas, mas há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

• Características clínicas dos sintomas;  
• Características da população;  
• Características econômicas da sociedade.  

Dessa forma, acessar os dados do PNAD-COVID-19 do IBGE (https://covid19.ibge.gov.br/pnad-covid/) e organizar esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

a. Utilização de no máximo 20 questionamentos realizados na pesquisa;  
b. Utilizar 3 meses para construção da solução;  
c. Caracterização dos sintomas clínicos da população;  
d. Comportamento da população na época da COVID-19;  
e. Características econômicas da Sociedade;  

Com objetivo de trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19

### Arquitetura de Dados

![Arquitetura](arquitetura2.jpeg)

1) Dados utilizado referente a Pesquisa Nacional por Amostra de Domicílios (PNAD COVID19) através do link [PNAD](https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=microdados&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19) *
2) Devido a volumetria dos dados, é feito o armazenamento no AWS S3 e usando camandas medalhão
3) Realizar o carregamento dos da camanda GOLD para o banco de dados PostgreSQL criado via AWS RDS
4) Construir a analise utilizando o Power BI conectado ao Data Mart

\* Os dados foram armazenados no GitHub para a construção desse trabalho

### **Observações**

#### **VS Code**

Para correto funcionamento, sempre reiniciar o Kernel antes de rodar o código

#### **Bibliotecas**

Para correto funcionamento, caso não tenha instalado as seguintes bibliotes: 

- `requests`
- `pandas`  
- `boto3`  
- `python-dotenv`  
- `SQLAlchemy`  
- `psycopg2-binary`  

Realizar a instalação via PIP INSTALL (https://pypi.org/) ou CONDA-FORGE (https://anaconda.org/conda-forge)

#### **Conexões**

Para realizar as conexões é utilizado um arquivo .env salvo no mesmo local do projeto e com a seguinte estrutura

```ini
# Credenciais do PostgreSQL PNAD
POSTGRES_USER_PNAD=
POSTGRES_PASSWORD_PNAD=
POSTGRES_HOST_PNAD=
POSTGRES_PORT_PNAD=5432
POSTGRES_DB_PNAD=

# Credenciais AWS
aws_access_key_id=
aws_secret_access_key=
aws_session_token=

region=us-east-1
output=json
```

#### **Configuração do PostgreSQL na AWS RDS**

##### 1. **Criar instância RDS com PostgreSQL (SandBox)**

1. Acesse o console AWS → [https://us-east-1.console.aws.amazon.com/rds/home?region=us-east-1#](https://console.aws.amazon.com/rds/)
2. Clique em **Criar banco de dados**
3. Selecione:
   - **Tipo de banco:** PostgreSQL
   - **Versão:** PostgreSQL 15 (ou mais recente)
   - **Modelo de uso:** SandBox
   - **Identificador da instância:** `postgres-pnad`
   - **Usuário:** `postgres`
   - **Senha:** crie uma senha segura
4. Tipo de instância: `db.t3.micro`
5. Armazenamento: 20 GB (SSD General Purpose)
6. **Acesso público:** Habilitado (Sim)
7. **Nome do banco de dados inicial:** `pnad_covid`
8. Clique em **Criar banco de dados**

##### **2. Liberar o IP na VPC / Grupo de Segurança (Security Group)**

1. Vá para **EC2 > Grupos de Segurança**
2. Encontre o grupo associado à instância RDS
3. Clique em **Editar regras de entrada**
4. Adicione uma nova regra:
   - Tipo: `PostgreSQL`
   - Porta: `5432`
   - Origem: `Seu IP` (ou `0.0.0.0/0` temporariamente para teste – cuidado com isso em produção)
5. Salve as alterações.

✅ Agora o acesso externo ao banco estará liberado para seu IP

**Obs.: Essa configuração será feita apenas uma vez no VPC Default**

### **Códigos**

#### **Importar bibliotecas**

In [1]:
# Importar biblioteca completa
import requests
import zipfile
import io
import pandas as pd
import os
import boto3
import sys
import psycopg2

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from botocore.exceptions import BotoCoreError, ClientError

#### **Funções (DEF)**

In [2]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("  -", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)


#### **Validar Conexões e criar engine banco de dados**

In [3]:
# Validar conexão com a AWS através do .env
load_dotenv()

try:
    sts_client = boto3.client(
        'sts',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
        region_name=os.getenv('AWS_REGION')
    )
    
    identity = sts_client.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAWIUHO6CVULDIIR3FL:user4308167=ricardviana1@gmail.com
Account: 430854566059
Arn: arn:aws:sts::430854566059:assumed-role/voclabs/user4308167=ricardviana1@gmail.com


In [4]:
# Criar a engine para conexão ao banco de dados usando
load_dotenv()

usuario = os.getenv("POSTGRES_USER_PNAD")
senha = os.getenv("POSTGRES_PASSWORD_PNAD")
host = os.getenv("POSTGRES_HOST_PNAD")
porta = os.getenv("POSTGRES_PORT_PNAD")
banco = os.getenv("POSTGRES_DB_PNAD")

engine = create_engine(f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}")

# Testar a conexão
test_connection(engine)

✅ Conectado com sucesso: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit
📄 Tabelas no banco:
  - questionario_pnad_covid


#### **Variaveis**

In [21]:
# Nome do bucket S3. subpastas e arquivo .parquet
s3_bucket = 'tech-challenge-fase3'
s3_subpasta = 'bronze'
s3_subpasta_silver = 'silver'
s3_subpasta_gold = 'gold'

BRONZE_PREFIX = f's3://{s3_bucket}/{s3_subpasta}/'
SILVER_PREFIX = f's3://{s3_bucket}/{s3_subpasta_silver}/'
GOLD_PREFIX = f's3://{s3_bucket}/{s3_subpasta_gold}/'

NOME_ARQUIVO_GOLD = 'pnad_final_tratado.parquet'

# Caminho do Github com dados do PNAD
api_url = 'https://api.github.com/repos/RicardViana/fiap-Big-Data/contents/PNAD-COVID/Microdados'

# Caminho do Github com dados do código IBGE UF
link_codigo_uf = 'https://raw.githubusercontent.com/RicardViana/tabela-uf-ibge/refs/heads/main/codigo_uf.csv'

# Caminho saida e entrada camada silver
caminho_saida_silver = SILVER_PREFIX + 'pnad_consolidado_enriquecido.parquet'
caminho_entrada_silver = SILVER_PREFIX + 'pnad_consolidado_enriquecido.parquet'

# Nome da tabela no PostgreSQL
nome_tabela = 'questionario_pnad_covid'
NOME_TABELA_PG = nome_tabela


#### **Configuração AWS e Banco de Dados**

In [ ]:
# Carregar as credencias .env
load_dotenv()

# Configuração storage_options
storage_options = {
    "key": os.getenv('AWS_ACCESS_KEY_ID'),
    "secret": os.getenv('AWS_SECRET_ACCESS_KEY'),
    "token": os.getenv('AWS_SESSION_TOKEN')
}

# Configuração S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
    region_name=os.getenv('AWS_REGION')
)

# Credenciais do PostgreSQL
usuario_pg = os.getenv("POSTGRES_USER_PNAD")
senha_pg = os.getenv("POSTGRES_PASSWORD_PNAD")
host_pg = os.getenv("POSTGRES_HOST_PNAD")
porta_pg = os.getenv("POSTGRES_PORT_PNAD")
banco_pg = os.getenv("POSTGRES_DB_PNAD")



#### **ETL**

In [24]:
# Criar Bucket e subpastas (camadas medalhões)
PASTAS_A_CRIAR = [s3_subpasta, s3_subpasta_silver, s3_subpasta_gold]
AWS_REGION = s3_client.meta.region_name

print(f"\nIniciando Processo de Validação e Criação no S3 ")

print(f"\nValidando o bucket '{s3_bucket}'")
bucket_pronto = False

try:
    s3_client.head_bucket(Bucket=s3_bucket)
    print(f"➡️  Bucket '{s3_bucket}' já existe")
    bucket_pronto = True

except ClientError as e:
    if e.response['Error']['Code'] == '404':
        print(f"Bucket '{s3_bucket}' não encontrado. Tentando criar")
        try:
            if AWS_REGION == "us-east-1":
                s3_client.create_bucket(Bucket=s3_bucket)
            else:
                location = {'LocationConstraint': AWS_REGION}
                s3_client.create_bucket(
                    Bucket=s3_bucket,
                    CreateBucketConfiguration=location
                )
            print(f"✅ Bucket '{s3_bucket}' criado com sucesso!")
            bucket_pronto = True
        except Exception as create_e:
            print(f"❌ Falha ao TENTAR CRIAR o bucket: {create_e}")
    else:
        print(f"❌ Erro de permissão ou outro problema ao verificar o bucket: {e}")

if bucket_pronto:
    print(f"\nValidando as subpastas no bucket '{s3_bucket}'")
    for nome_pasta in PASTAS_A_CRIAR:
        chave_pasta = nome_pasta if nome_pasta.endswith('/') else nome_pasta + '/'
        
        try:
            s3_client.head_object(Bucket=s3_bucket, Key=chave_pasta)
            print(f"  ➡️  Pasta '{chave_pasta}' já existe")

        except ClientError as e:
            if e.response['Error']['Code'] == '404':
                try:
                    s3_client.put_object(
                        Bucket=s3_bucket,
                        Key=chave_pasta,
                        Body=''
                    )
                    print(f"  ✅ Pasta '{chave_pasta}' criada com sucesso")
                except Exception as create_e:
                    print(f"  ❌ Falha ao TENTAR CRIAR a pasta '{chave_pasta}': {create_e}")
            else:
                print(f"  ❌ Erro ao verificar a pasta '{chave_pasta}': {e}")
else:
    print("\nCriação das pastas abortada, pois houve um problema com o bucket")

print("\nProcesso Finalizado")


Iniciando Processo de Validação e Criação no S3 

Validando o bucket 'tech-challenge-fase3'
➡️  Bucket 'tech-challenge-fase3' já existe

Validando as subpastas no bucket 'tech-challenge-fase3'
  ➡️  Pasta 'bronze/' já existe
  ➡️  Pasta 'silver/' já existe
  ➡️  Pasta 'gold/' já existe

Processo Finalizado


In [ ]:
# Carregar os dados para camada Bronze

# Realizar a consulta a URL
print(f"Buscando lista de arquivos em: {api_url}")
response = requests.get(api_url)
response.raise_for_status()
files = response.json()

# Itera sobre cada item no diretório do GitHub
for file_info in files:

    if file_info['name'].endswith('.zip'):
        zip_name = file_info['name']
        zip_url = file_info['download_url']

        print(f"\nProcessando: {zip_name}")
        
        try:
            r_zip = requests.get(zip_url)
            r_zip.raise_for_status()
            zip_content = io.BytesIO(r_zip.content)

            with zipfile.ZipFile(zip_content) as z:
                csv_filename = [f for f in z.namelist() if f.endswith('.csv')][0]

                print(f"Arquivo CSV encontrado: {csv_filename}")

                csv_content_bytes = z.read(csv_filename)
                caminho_completo_s3 = f"{s3_subpasta}/{csv_filename}"

                print(f"Enviando para o S3 em: '{caminho_completo_s3}'")
                s3_client.put_object(
                    Bucket=s3_bucket,
                    Key=caminho_completo_s3,
                    Body=csv_content_bytes
                )

                print(f"✅ Sucesso! Arquivo enviado para s3://{s3_bucket}/{caminho_completo_s3}")
        
        except Exception as e:
            print(f"Erro: Ocorreu um erro inesperado ao processar {zip_name}. Erro: {e}")
            sys.exit()
            
print("\nProcesso para o S3 concluído!")

In [ ]:
# Carregar dados para camada silver

# Ler arquivos CSV PNAD COVID do S3, consolidar e gerar Data Frames
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix='bronze/')
bronze_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.csv')]
lista_de_dataframes = []

print("Lendo arquivos da camada Bronze no S3")

for file_key in bronze_files:

    if 'codigo_uf' in file_key:
        continue
    
    file_path = f"s3://{s3_bucket}/{file_key}"
    print(f"Lendo {file_path}")
    df_temp = pd.read_csv(file_path, storage_options=storage_options, sep=',')
    lista_de_dataframes.append(df_temp)

df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)
print(f"\nConsolidação concluída. {len(df_consolidado)} linhas lidas da camada Bronze")

# Ler arquivos CSV Codigo IBGE e gerar Data Frame
print(f"\nLendo tabela com os códigos do IBGE de: {link_codigo_uf}")
df_uf = pd.read_csv(link_codigo_uf, sep=",")
print("Data Frame criado com sucesso")

# Relacionar os Data Frame
print("\nIniciando enriquecimento dos dados (merge com UF)")
df_silver = pd.merge(
    df_consolidado,
    df_uf,
    how='left',
    left_on='UF', 
    right_on='Código'
)

df_silver = df_silver.drop(columns=["Código"])
df_silver = df_silver.rename(columns={"UF_x": "UF", "UF_y": "UF_Nome"})
print("Enriquecimento concluído")

# Ver o resultado final
print("\nData Frame com os primeiros registros")
display(df_silver.head(5))

print("Data Frame com os ultimos registros")
display(df_silver.tail(5))


In [ ]:
# Salvar dados
print(f"\nSalvando dados da camada Silver em: {caminho_saida_silver}")

df_silver.to_parquet(
    caminho_saida_silver,
    index=False,
    storage_options=storage_options
)

print("✅ Sucesso! Camada Silver criada e salva no S3")

In [ ]:
# Tratar os dados para a gold
print(f"Lendo dados da camada Silver de: {caminho_entrada_silver}")

try:
    df_silver = pd.read_parquet(caminho_entrada_silver, storage_options=storage_options)
    print(f"Leitura concluída. DataFrame com {len(df_silver)} linhas e {len(df_silver.columns)} colunas.")

except Exception as e:
    print(f"❌ ERRO ao ler o arquivo da camada Silver. Verifique o caminho e as permissões. Detalhes: {e}")
    sys.exit()

# Copiar e padronizar colunas para minúsculas
df_estruturado = df_silver.copy()
df_estruturado.columns = df_estruturado.columns.str.lower()

# -----------------------------
# Colunas fixas (com descrição)
# -----------------------------
colunas_fixas = [
    'ano',       # Ano
    'v1013',     # Mês do Ano
    'v1012',     # Semana do Mês
    'uf',        # Sigla da Unidade da Federação
    'capital',   # Capital do Estado
    'rm_ride',   # Região Metropolitana e Região Administrativa Integrada de Desenvolvimento
]

# --------------------------------------
# Colunas desejadas (com descrição)
# --------------------------------------
colunas_desejadas = [
    'a002',      # Idade
    'a003',      # Sexo
    'a004',      # Raça ou Cor
    'a006b',     # Você está tendo aulas presenciais?
    'b008',      # O(A) Sr(a) fez algum teste para saber se estava infectado(a) pelo coronavírus?
    'b009a',     # Fez o exame coletado com cotonete na boca e/ou nariz (SWAB)?
    'b009c',     # Fez o exame de coleta de sangue através de furo no dedo?
    'b009e',     # Fez o exame de coleta de sangue através da veia do braço?
    'a005',      # Escolaridade
    'a006',      # Frequenta escola
    'a006a',     # A escola/faculdade que frequenta é pública ou privada?
    'b0011',     # Na semana passada teve febre?
    'b0012',     # Na semana passada teve tosse?
    'b0013',     # Na semana passada teve dor de garganta?
    'b0014',     # Na semana passada teve dificuldade para respirar?
    'b0015',     # Na semana passada teve dor de cabeça?
    'b0016',     # Na semana passada teve dor no peito?
    'b0017',     # Na semana passada teve náusea?
    'b0018',     # Na semana passada teve nariz entupido ou escorrendo?
    'b0019',     # Na semana passada teve fadiga?
    'b00110',    # Na semana passada teve dor nos olhos?
    'b00111',    # Na semana passada teve perda de cheiro ou sabor?
    'b00112',    # Na semana passada teve dor muscular?
    'b00113',    # Na semana passada teve diarreia?
]

# Unir listas mantendo a ordem e sem duplicar
todas_colunas = list(dict.fromkeys([*colunas_fixas, *colunas_desejadas]))

# Manter apenas as colunas que existem no DataFrame após padronização
colunas_existentes = [c for c in todas_colunas if c in df_estruturado.columns]

# -----------------------------
# Filtro: 3 últimos valores de v1013
# -----------------------------
if 'v1013' in df_estruturado.columns:
    # Coletar valores únicos (excluindo NaN), ordenar e pegar os 3 últimos
    ultimos_3 = sorted(pd.unique(df_estruturado['v1013'].dropna()))[-3:]

    # Filtrar registros pertencentes aos 3 últimos meses e selecionar colunas
    estrutura_final = (
        df_estruturado[df_estruturado['v1013'].isin(ultimos_3)][colunas_existentes]
        .reset_index(drop=True)
    )
else:
    # Se não houver v1013, apenas seleciona as colunas existentes
    estrutura_final = df_estruturado[colunas_existentes].reset_index(drop=True)

# (Opcional) Avisos sobre colunas ausentes
faltantes = [c for c in todas_colunas if c not in df_estruturado.columns]
if faltantes:
    print(f"Atenção: estas colunas não foram encontradas e ficaram de fora: {faltantes}")

# Ver o resultado final
print("\nData Frame com os primeiros registros")
display(estrutura_final.head(5))

print("Data Frame com os ultimos registros")
display(estrutura_final.tail(5))


In [ ]:
# Carregar os dados para a gold
caminho_saida_gold = GOLD_PREFIX + 'pnad_final_tratado.parquet'
print(f"\nSalvando dados da camada Gold em: {caminho_saida_gold}")

estrutura_final.to_parquet(
    caminho_saida_gold,
    index=False,
    storage_options=storage_options
)

print(f"✅ Sucesso! Camada Gold criada com {len(estrutura_final)} linhas e salva no S3.")

In [ ]:
# Criar a tabela
print("Iniciando a criação do esquema da tabela")

try:
    df_schema = estrutura_final.head(0)
    df_schema.to_sql(
        nome_tabela, 
        con=engine, 
        if_exists='replace', 
        index=False)
    
    print(f"✅ Esquema da tabela '{nome_tabela}' criado com sucesso no PostgreSQL!")

except Exception as e:
    print(f"❌ Erro na Parte 1: {e}")
    sys.exit()

In [ ]:
# Carregar dados da Gold para PostgreSQL

conn = None

try:
    # Passo 1: Ler os dados da camada Gold do S3
    caminho_completo_gold = GOLD_PREFIX + NOME_ARQUIVO_GOLD
    print(f"Lendo dados da camada Gold de: {caminho_completo_gold}")
    df_gold = pd.read_parquet(caminho_completo_gold, storage_options=storage_options)
    print(f"Leitura concluída. DataFrame com {len(df_gold)} linhas pronto para carregar.")

    # Passo 2: Conectar ao PostgreSQL usando psycopg2
    print("\nConectando ao banco de dados PostgreSQL")
    conn = psycopg2.connect(
        host=host_pg, database=banco_pg, user=usuario_pg, password=senha_pg, port=porta_pg
    )
    cursor = conn.cursor()
    print("Conexão bem-sucedida!")

    # Passo 3 (Opcional, mas recomendado): Limpar a tabela antes da carga.
    # TRUNCATE é mais rápido que DROP/CREATE para limpar uma tabela.
    print(f"Limpando a tabela de destino '{NOME_TABELA_PG}'")
    cursor.execute(f"TRUNCATE TABLE {NOME_TABELA_PG}")
    
    # Passo 4: Preparar o DataFrame para o formato do COPY
    # Convertemos o DataFrame para um arquivo CSV em memória (buffer)
    buffer = io.StringIO()
    df_gold.to_csv(buffer, index=False, header=False, sep='\t')
    buffer.seek(0)

    # Passo 5: Executar o COPY de alta performance
    print(f"Iniciando a carga de {len(df_gold)} linhas via COPY")
    cursor.copy_from(buffer, NOME_TABELA_PG, sep='\t', null='')
    print("Carga via COPY concluída.")

    # Passo 6: Efetivar a transação no banco
    conn.commit()
    print(f"✅ Sucesso! Transação efetivada. Dados carregados na tabela '{NOME_TABELA_PG}'.")

except Exception as e:
    if conn:
        conn.rollback() # Se der erro, desfaz a transação
    print(f"❌ Ocorreu um erro durante o processo de carga: {e}")

finally:
    if conn:
        conn.close() # Garante que a conexão seja sempre fechada
        print("Conexão com PostgreSQL fechada.")